##Ephemeris Generation##

This notebook is intended to test various methods of generating the positions and velocities of moving objects, including testing the accuracy of the resulting values calculated by interpolating between the points calculated by an n-body integrator. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read some orbital parameters.
import pandas
orbitfile = 'pha20141031.des'

orbits = pandas.read_table(orbitfile, sep='\s', engine='python')
orbits = orbits.to_records()

In [3]:
# Set up to run OpenOrb to generate ephemerides.
from itertools import repeat
import pyoorb as oo

oo.pyoorb.oorb_init(ephemeris_fname="")

3

Set up the times at which to calculate 'rough' positions. 
We will interpolate between these times to find the positions at the actual time of a visit. 

In [4]:
timestep = 1 # days
timestart = 49353
nyears = 1 # years
timeend = timestart + 365 * nyears + 1.0
times = np.arange(timestart, timeend + timestep/2.0, timestep)
# For pyoorb, we need to tag times with timescales;
# 1= MJD_UTC, 2=UT1, 3=TT, 4=TAI
ephTimes = np.array(zip(times, repeat(4, len(times))), dtype='double', order='F')
print ephTimes.shape

(367, 2)


Python oorb element format: 
        # Translate orbital elements into array that pyoorb will like.
        # PyOrb wants ::
        # 0: orbitId  (cannot be a string)
        # 1 - 6: orbital elements, using radians for angles
        # 7: element type code, where 2 = cometary - means timescale is TT, too
        # 8: epoch
        # 9: timescale for the epoch; 1= MJD_UTC, 2=UT1, 3=TT, 4=TAI
        # 10: magHv
        # 11: G
so we have to do a little translating from the orbits DataFrame to the elements we want in this array. 

In [56]:
def packOorbElem(sso):
    """
    Convert an array of orbital elements into the array OpenOrb needs as input.
    'sso' can be the orbital elements of a single object or of multiple objects.
    """
    if len(sso.shape) == 0:
        nSso = 0
        sso0 = sso
        elem_type = 2
        epoch_type = 3
        Hval = 20.0
        phaseV = 0.15
    else:
        nSso = len(sso)
        sso0 = sso[0]
        elem_type = np.zeros(nSso) + 2
        epoch_type = np.zeros(nSso) + 3
        Hval = np.zeros(nSso) + 20.0
        phaseV= np.zeros(nSso) + 0.15
    # Also, the orbitID has to be a float, rather than a string, so substitute if needed.
    objid = None
    if '!!ObjID' in sso.dtype.names:
        objid = '!!ObjID'
    elif 'objid' in sso.dtype.names:
        objid = 'objid'
    elif '!!OID' in sso.dtype.names:
        objid = '!!OID'
    if (objid is not None) and (isinstance(sso0[objid], float) or isinstance(sso0[objid], int)):
        orbids = sso[objid]
    else:
        if nSso == 0:
            orbids = 0
        else:
            orbids = np.arange(0, nSso, 1)
    # Search for appropriate 'keywords' for some other orbital parameters that vary in different input formats
    BigOmega = 'Omega/node'
    if 'node' in sso.dtype.names:
        BigOmega = 'node'
    omega = 'omega/argperi'
    if 'argperi' in sso.dtype.names:
        omega = 'argperi'
    tperi = 't_p'
    if 'timeperi' in sso.dtype.names:
        tperi = 'timeperi'
    inc = 'i'
    if 'inc' in sso.dtype.names:
        inc = 'inc'
    epoch = 't_0'
    if 'epoch' in sso.dtype.names:
        epoch = 'epoch'
    magH = 'magHv'
    if 'H' in sso.dtype.names:
        magH = 'H'
    if magH in sso.dtype.names:
        Hval = sso[magH]
    if 'phaseV' in sso.dtype.names:
        phaseV = sso['phaseV']
    
    # Convert to format for pyoorb, INCLUDING converting inclination, node, argperi to RADIANS
    oorbArray = np.column_stack((orbids, sso['q'], sso['e'], np.radians(sso[inc]),
                                 np.radians(sso[BigOmega]), np.radians(sso[omega]),
                                 sso[tperi], elem_type, sso[epoch], epoch_type, 
                                 Hval, phaseV))                                            
    return oorbArray

In [45]:
def unpackEphs(oorbephem):
    if 
    # then swap time/ephemeris element, so that we can get easy arrays of things like 'ra', 'dec'.
    oorbephem = np.swapaxes(oorbephem, 0, 1)
    dist = oorbephem[0]
    ra = oorbephem[1]
    dec = oorbephem[2]
    magV = oorbephem[3]
    time = oorbephem[4]
    dradt = oorbephem[6]
    ddecdt = oorbephem[7]
    phaseangle = oorbephem[8]
    solarelon = oorbephem[9]
    velocity = np.sqrt(dradt**2 + ddecdt**2)
    ephs = np.rec.fromarrays([time, ra, dec, dradt, ddecdt, dist, magV, phaseangle, solarelon, velocity], 
                             names=['time', 'ra', 'dec', 'dradt', 'ddecdt', 'dist', 
                                    'magV', 'phaseangle', 'solarelon', 'velocity'])
    return ephs
    

In [64]:
%%timeit
orbsubset = orbits[0:30]
oorbelems = packOorbElem(orbsubset)
oorbephs, err = oo.pyoorb.oorb_ephemeris(in_orbits = oorbelems, in_obscode=807, in_date_ephems=ephTimes)

1 loops, best of 3: 2.35 s per loop


In [65]:
%%timeit
for sso in orbsubset:
    orbelems = packOorbElem(sso)
    oorbephs, err = oo.pyoorb.oorb_ephemeris(in_orbits = oorbelems, in_obscode=807, in_date_ephems=ephTimes)

1 loops, best of 3: 320 ms per loop
